# DeepSphere using SHREC17 dataset
Projection of 3D objects on a unit sphere.
This sphere use a HEALpix sampling, and several features are collected:
* projection ray length
* cos/sin with surface normal
* same features using the convex hull

## 0 Load libs

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import shutil
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import time
import matplotlib.pyplot as plt

In [3]:
from deepsphere import models, experiment_helper, plot, utils
from deepsphere.data import LabeledDatasetWithNoise, LabeledDataset
import hyperparameters

from SHREC17.load_shrec import Shrec17DeepSphere as shrecDataset

# 0.2 Define parameters

In [4]:
Nside = 32
experiment_type = 'FCN' # 'CNN'
ename = '_'+experiment_type
datapath = '../data/shrec17/'

In [5]:
sigma_noise = 0
noise_dataset = True
augmentation = 1    # number of element per file (1 = no augmentation of dataset)

## 1 Load dataset

download dataset if True, preprocess data and store it in npy files, and load it in a dataset object

In [9]:
download = False
train_dataset = shrecDataset(datapath, 'train', perturbed=noise_dataset, download=download, nside=Nside, augmentation=augmentation, nfile=-1)


31363it [00:40, 771.86it/s]


## 1.1 Preprocess the dataset

In [10]:
x_train, labels_train, x_val, labels_val = train_dataset.return_data(train=True)

Number of elements / class
  Training set: 
    Class 0: 1982 elements
    Class 1: 160 elements
    Class 2: 42 elements
    Class 3: 55 elements
    Class 4: 423 elements
    Class 5: 126 elements
    Class 6: 889 elements
    Class 7: 38 elements
    Class 8: 200 elements
    Class 9: 234 elements
    Class 10: 82 elements
    Class 11: 452 elements
    Class 12: 758 elements
    Class 13: 53 elements
    Class 14: 53 elements
    Class 15: 26 elements
    Class 16: 1744 elements
    Class 17: 257 elements
    Class 18: 3235 elements
    Class 19: 330 elements
    Class 20: 37 elements
    Class 21: 45 elements
    Class 22: 518 elements
    Class 23: 33 elements
    Class 24: 368 elements
    Class 25: 150 elements
    Class 26: 388 elements
    Class 27: 77 elements
    Class 28: 268 elements
    Class 29: 193 elements
    Class 30: 1131 elements
    Class 31: 227 elements
    Class 32: 793 elements
    Class 33: 50 elements
    Class 34: 34 elements
    Class 35: 70 elements
    

In [11]:
nclass = train_dataset.nclass
num_elem = train_dataset.N
ids = train_dataset.retrieve_ids()
print('number of class:',nclass,'\nnumber of elements:',num_elem,'\nfirst id:',ids[0])

number of class: 55 
number of elements: 31363 
first id: 000003


## 2 Classification using DeepSphere

In [12]:
training = LabeledDatasetWithNoise(x_train, labels_train, end_level=sigma_noise)
validation = LabeledDataset(x_val, labels_val)

In [13]:
EXP_NAME = 'shrec17_40sim_{}sides_{}noise{}'.format(Nside, sigma_noise, ename)

In [14]:
params = hyperparameters.get_params_shrec17(training.N, EXP_NAME, Nside, train_dataset.nclass, architecture=experiment_type)
model = models.deepsphere(**params)

#sides: [32, 16, 8, 4, 2, 2]
#pixels: [12288, 3072, 768, 192, 48, 48]
#samples per batch: 32
=> #pixels per batch (input): 393,216
=> #pixels for training (input): 24,664,473,600
Learning rate will start at 2.0e-04 and finish at 1.1e-31.
NN architecture
  input: M_0 = 12288
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 12288 * 96 / 4 = 294912
    weights: F_0 * F_1 * K_1 = 1 * 96 * 5 = 480
    biases: F_1 = 96
    batch normalization
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 3072 * 192 / 4 = 147456
    weights: F_1 * F_2 * K_2 = 96 * 192 * 5 = 92160
    biases: F_2 = 192
    batch normalization
  layer 3: cgconv3
    representation: M_2 * F_3 / p_3 = 768 * 384 / 4 = 73728
    weights: F_2 * F_3 * K_3 = 192 * 384 * 5 = 368640
    biases: F_3 = 384
    batch normalization
  layer 4: cgconv4
    representation: M_3 * F_4 / p_4 = 192 * 384 / 4 = 18432
    weights: F_3 * F_4 * K_4 = 384 * 384 * 5 = 737280
    biases: F_4 = 384
    batch normalization
  layer 5: cgc

In [15]:
shutil.rmtree('summaries/{}/'.format(EXP_NAME), ignore_errors=True)
shutil.rmtree('checkpoints/{}/'.format(EXP_NAME), ignore_errors=True)

## 2.2 Train Network

In [ ]:
accuracy_validation, loss_validation, loss_training, t_step = model.fit(training, validation)

step 313 / 62725 (epoch 0.40 / 80):
  learning_rate = 1.07e-04, training loss = 1.09e+00
{2, 3, 5, 7, 13, 15, 20, 21, 25, 29, 33, 34, 35, 38, 39, 40, 42, 43, 45, 48, 50, 51}
  validation accuracy: 72.76 (4564 / 6273), f1 (weighted): 69.25, loss: 1.04e+00
  CPU time: 111s, wall time: 183s


/home/gusset/miniconda3/envs/PDMsphere/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


step 626 / 62725 (epoch 0.80 / 80):
  learning_rate = 5.72e-05, training loss = 8.00e-01
{33, 2, 34, 35, 5, 7, 39, 42, 43, 13, 45, 15, 48, 51, 20, 21, 54, 25}
  validation accuracy: 75.23 (4719 / 6273), f1 (weighted): 72.95, loss: 9.13e-01
  CPU time: 213s, wall time: 357s
step 939 / 62725 (epoch 1.20 / 80):
  learning_rate = 3.06e-05, training loss = 8.00e-01
{33, 2, 34, 35, 7, 42, 43, 13, 45, 15, 51, 20}
  validation accuracy: 77.27 (4847 / 6273), f1 (weighted): 75.21, loss: 8.43e-01
  CPU time: 314s, wall time: 530s
step 1252 / 62725 (epoch 1.60 / 80):
  learning_rate = 1.63e-05, training loss = 1.14e+00
{33, 2, 3, 34, 35, 38, 7, 39, 42, 43, 13, 45, 15, 48, 21}
  validation accuracy: 77.32 (4850 / 6273), f1 (weighted): 75.22, loss: 8.15e-01
  CPU time: 414s, wall time: 704s
step 1565 / 62725 (epoch 2.00 / 80):
  learning_rate = 8.74e-06, training loss = 7.91e-01
{33, 2, 34, 35, 5, 38, 7, 42, 43, 13, 45, 15, 48, 21, 54, 25}
  validation accuracy: 78.35 (4915 / 6273), f1 (weighted): 7

In [ ]:
plot.plot_loss(loss_training, loss_validation, t_step, params['eval_frequency'])

In [ ]:
predictions, loss = model.predict(x_train, labels_train)

In [ ]:
predictions-labels_train

In [ ]:
predictions, loss = model.predict(x_val, labels_val)

In [ ]:
predictions-labels_val

In [34]:
model.evaluate(x_val, labels_val)

/mnt/scratch/students/gusset/DeepSphere/deepsphere/../checkpoints/shrec17_40sim_32sides_0noise_FCN
INFO:tensorflow:Restoring parameters from /mnt/scratch/students/gusset/DeepSphere/deepsphere/../checkpoints/shrec17_40sim_32sides_0noise_FCN/model-1750
{2, 5, 37, 7, 38, 42, 43, 46, 15, 19, 51, 21, 23, 27, 28, 29}


/home/gusset/miniconda3/envs/PDMsphere/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gusset/miniconda3/envs/PDMsphere/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


('accuracy: 70.00 (210 / 300), f1 (weighted): 66.63, loss: 1.48e+00\nCPU time: 1s, wall time: 1s',
 70.0,
 66.63168219425292,
 1.4809802881876628)